---

_You are currently looking at **version 1.5** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

# Assignment 3 - More Pandas
This assignment requires more individual learning then the last one did - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

### Question 1 (20%)
Load the energy data from the file `Energy Indicators.xls`, which is a list of indicators of [energy supply and renewable electricity production](Energy%20Indicators.xls) from the [United Nations](http://unstats.un.org/unsd/environment/excel_file_tables/2013/Energy%20Indicators.xls) for the year 2013, and should be put into a DataFrame with the variable name of **energy**.

Keep in mind that this is an Excel file, and not a comma separated values file. Also, make sure to exclude the footer and header information from the datafile. The first two columns are unneccessary, so you should get rid of them, and you should change the column labels so that the columns are:

`['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']`

Convert `Energy Supply` to gigajoules (there are 1,000,000 gigajoules in a petajoule). For all countries which have missing data (e.g. data with "...") make sure this is reflected as `np.NaN` values.

Rename the following list of countries (for use in later questions):

```"Republic of Korea": "South Korea",
"United States of America": "United States",
"United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
"China, Hong Kong Special Administrative Region": "Hong Kong"```

There are also several countries with numbers and/or parenthesis in their name. Be sure to remove these, 

e.g. 

`'Bolivia (Plurinational State of)'` should be `'Bolivia'`, 

`'Switzerland17'` should be `'Switzerland'`.

<br>

Next, load the GDP data from the file `world_bank.csv`, which is a csv containing countries' GDP from 1960 to 2015 from [World Bank](http://data.worldbank.org/indicator/NY.GDP.MKTP.CD). Call this DataFrame **GDP**. 

Make sure to skip the header, and rename the following list of countries:

```"Korea, Rep.": "South Korea", 
"Iran, Islamic Rep.": "Iran",
"Hong Kong SAR, China": "Hong Kong"```

<br>

Finally, load the [Sciamgo Journal and Country Rank data for Energy Engineering and Power Technology](http://www.scimagojr.com/countryrank.php?category=2102) from the file `scimagojr-3.xlsx`, which ranks countries based on their journal contributions in the aforementioned area. Call this DataFrame **ScimEn**.

Join the three datasets: GDP, Energy, and ScimEn into a new dataset (using the intersection of country names). Use only the last 10 years (2006-2015) of GDP data and only the top 15 countries by Scimagojr 'Rank' (Rank 1 through 15). 

The index of this DataFrame should be the name of the country, and the columns should be ['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations',
       'Citations per document', 'H index', 'Energy Supply',
       'Energy Supply per Capita', '% Renewable', '2006', '2007', '2008',
       '2009', '2010', '2011', '2012', '2013', '2014', '2015'].

*This function should return a DataFrame with 20 columns and 15 entries.*

In [1]:
import pandas as pd
import numpy as np

enr = pd.read_excel('Energy Indicators.xls', skiprows=17, skip_footer=38)
enr.name = 'energy'
enr.drop(enr.columns[[0, 1]], axis=1, inplace=True)
enr.columns = ['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']
enr['Energy Supply'] = enr['Energy Supply'].astype(str).str.replace('\.\.\.', 'NaN').astype(float)
enr['Energy Supply per Capita'] = enr['Energy Supply per Capita'].astype(str).str.replace('\.\.\.', 'NaN').astype(float)
enr['Energy Supply'] = enr['Energy Supply'].apply(lambda x: x * 1000000)
enr['Country'] = enr['Country'].str.replace('^Republic of Korea$', 'South Korea')
enr['Country'] = enr['Country'].str.replace('United States of America', 'United States')
enr['Country'] = enr['Country'].str.replace('United Kingdom of Great Britain and Northern Ireland', 'United Kingdom')
enr['Country'] = enr['Country'].str.replace('China, Hong Kong Special Administrative Region', 'Hong Kong')
enr['Country'] = enr['Country'].str.replace('\s*\(.*?\)\s*', '').str.replace('\d+','')

gdp = pd.read_csv('world_bank.csv', skiprows=4)
gdp.name = 'gdp'
gdp['Country Name'] = gdp['Country Name'].str.replace('Korea, Rep.', 'South Korea')
gdp['Country Name'] = gdp['Country Name'].str.replace('Iran, Islamic Rep.', 'Iran')
gdp['Country Name'] = gdp['Country Name'].str.replace('Hong Kong SAR, China', 'Hong Kong')
gdp=gdp.rename(columns = {'Country Name':'Country'})
aux = range(1960, 2006)
drop = [str(i) for i in aux]
gdp = gdp.drop(drop, axis=1)

scim = pd.read_excel('scimagojr-3.xlsx')
scim.name = 'ScimEn'

def answer_one():
    df = gdp.merge(scim, how='inner', right_on='Country', left_on='Country')
    df = df.merge(enr, right_on='Country', left_on='Country')
    df = df.set_index('Country')
    df = df.sort_values(by='Rank')[:15]
    df = df[['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations', 'Citations per document', 'H index', 'Energy Supply', 'Energy Supply per Capita', '% Renewable', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']]
    
    return df

answer_one().shape

(15, 20)

In [2]:
energy = enr

energy['original'] = pd.read_excel('Energy Indicators.xls',
                                   usecols=[1],
                                   index_col=0).loc['Afghanistan':'Zimbabwe'].index.tolist()
enDict = {
    "Republic of Korea": "South Korea",
    "United States of America": "United States",
    "United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
    "China, Hong Kong Special Administrative Region": "Hong Kong"
}
res = ''
# check for trailing whitespace
tws = energy[energy['Country'].str.contains('\s+$')]
res += "Trailing whitespace Test: "
res += ['Passed\n', 'Failed\n {}\n'.format(tws['Country'])][len(tws) > 0]

# check parentheses ()
prth = energy[
    (energy['Country'].str.contains('\(')) | (
        energy['Country'].str.contains('\)'))]
res += "Parentheses Test: "
res += ['Passed\n', 'Failed\n {}\n'.format(prth['Country'])][len(prth) > 0]

# check footnote digits
dg = energy[energy['Country'].str.contains('\d+$')]
res += "Digits Test: "
res += ['Passed\n', 'Failed\n {}\n'.format(dg['Country'])][len(dg) > 0]

res += 'Replace Countries Test: '
tested = energy.loc[energy['Country'].isin(
    list(enDict.values())), 'Country'].sort_values().values.tolist()
res += ['Failed\n', 'Passed\n'][tested == list(sorted(enDict.values()))]

res += 'regex use Test: '

encodedC = '11,7,7,14,7,6,8,19,9,7,5,9,7,10,7,7,10,8,7,7,6,5,7,6,7,32,22,8,6,22,17,8,12,7,10,8,8,6,14,24,4,5,5,9,42,8,7,5,12,10,13,7,4,7,6,14,37,32,7,8,8,18,7,5,11,17,7,7,8,14,16,4,7,6,13,16,5,6,7,7,5,9,6,9,7,10,4,9,8,6,13,6,5,8,7,7,5,9,4,4,7,11,6,5,7,5,6,6,10,5,8,6,10,32,6,7,7,7,5,13,9,10,10,6,8,8,4,5,16,10,10,9,6,10,8,10,10,7,10,7,7,5,5,11,13,11,9,5,7,4,24,6,4,8,5,6,16,8,4,11,6,8,11,5,11,19,7,7,18,6,12,21,11,25,32,5,21,12,7,6,10,12,9,12,8,8,15,7,12,11,5,9,18,5,8,9,6,11,20,10,8,41,11,4,5,19,7,6,12,24,6,6,7,20,14,27,13,28,7,10,7,9,8,25,5,6,8'
energy['tested'] = energy['Country'].apply(len)
energy['actual'] = encodedC.split(',')
energy['actual'] = energy['actual'].astype(int)
res += ['Failed\n', 'Passed\n'][energy['tested'].equals(energy['actual'])]
if not energy['tested'].equals(energy['actual']):
    res += 'Mismatched countries:\n'
    mismatch = energy.loc[energy['tested'] != (energy['actual']), [
        'original', 'Country', 'tested', 'actual']].values.tolist()
    res += '\n'.join('"{:}" miss-cleaned as  "{:}"'.format(o, r)
                     for o, r, s, v in mismatch)

print(res)

Trailing whitespace Test: Passed
Parentheses Test: Passed
Digits Test: Passed
Replace Countries Test: Passed
regex use Test: Failed
Mismatched countries:



In [3]:
GDP = gdp.filter(regex='^Country$|^Country Name$|200[6-9]|201[0-5]')
gdpCols = [str(c) for c in range(2006, 2016)] + ['Country']
tested = GDP.columns.sort_values().tolist()
res = 'GDP columns length: \n'
res += ['Failed\n', 'Passed\n'][len(gdpCols) == len(tested) ]

res += 'GDP columns test:'
res += ['Failed\n', 'Passed\n'][gdpCols == tested ]
if gdpCols != tested:
    
    res += 'mismatched columns:\n'
    res += '\n'.join('\t\t"{:}" should be "{:}"'.format(a, t)
                     for a, t in zip(gdpCols, tested) if a != t)
else:
    encodedC = '5,7,11,6,7,10,20,9,7,14,19,9,7,10,7,7,5,12,10,8,7,12,22,7,6,7,7,6,8,17,6,8,24,6,30,11,15,5,5,13,8,11,8,7,10,10,22,4,7,14,6,14,7,8,8,7,18,7,43,26,19,45,21,7,16,9,7,5,7,8,14,40,7,4,6,13,21,5,14,7,5,9,6,11,13,17,6,7,9,9,4,6,11,9,8,38,7,5,7,9,16,9,9,9,8,11,5,14,7,4,4,7,6,5,7,6,5,10,5,15,8,8,19,11,6,6,49,7,7,7,5,9,25,44,10,13,9,19,19,7,25,9,10,6,16,24,7,6,7,10,8,26,6,16,13,14,4,5,7,50,10,8,24,10,10,9,6,8,13,7,13,5,7,9,11,6,5,5,11,12,4,18,8,6,4,11,5,16,6,24,11,25,8,8,27,25,16,5,7,18,6,10,12,5,7,9,15,12,11,10,7,6,42,11,18,12,21,8,15,8,6,9,25,10,20,24,4,42,44,4,8,10,12,52,49,11,5,23,41,19,7,6,6,8,6,7,19,7,13,10,30,13,22,21,7,7,18,5,5,11,12,16,6,8'
    res += '\nCountry Cleaning Test:\n'
    GDP['original'] = pd.read_csv('world_bank.csv',
                                  usecols=[0],
                                  index_col=0).loc['Aruba':'Zimbabwe'].index.tolist()
    GDP['tested'] = GDP['Country'].apply(len)
    GDP['actual'] = encodedC.split(',')
    GDP['actual'] = GDP['actual'].astype(int)

    res += ['Failed\n', 'Passed\n'][GDP['tested'].equals(GDP['actual'])]
    if not GDP['tested'].equals(GDP['actual']):
        res += 'Mismatched countries:\n'
        mismatch = GDP.loc[GDP['tested'] != (GDP['actual']), [
            'original', 'Country', 'tested', 'actual']].values.tolist()
        res += '\n'.join('"{:}" miss-cleaned as  "{:}"'.format(o, r)
                         for o, r, s, v in mismatch)

print(res)

GDP columns length: 
Passed
GDP columns test:Passed

Country Cleaning Test:
Failed
Mismatched countries:



In [4]:
ScimEn = scim
res= 'Shape Test:'
res += ['Failed\n', 'Passed\n'][len(ScimEn)==191]
print(res) 

Shape Test:Passed



### Question 2 (6.6%)
The previous question joined three datasets then reduced this to just the top 15 entries. When you joined the datasets, but before you reduced this to the top 15 items, how many entries did you lose?

*This function should return a single number.*

In [5]:
%%HTML
<svg width="800" height="300">
  <circle cx="150" cy="180" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="blue" />
  <circle cx="200" cy="100" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="red" />
  <circle cx="100" cy="100" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="green" />
  <line x1="150" y1="125" x2="300" y2="150" stroke="black" stroke-width="2" fill="black" stroke-dasharray="5,3"/>
  <text  x="300" y="165" font-family="Verdana" font-size="35">Everything but this!</text>
</svg>

In [6]:
def answer_two():
    set1 = set(enr['Country'].values)
    set2 = set(gdp['Country'].values)
    set3 = set(scim['Country'].values)
    
    u = set1.union(set2).union(set3)
    print(len(u))
    i = set1.intersection(set2).intersection(set3)
    print(len(i))
    
    return len(u-i)
    
answer_two()

318
162


156

<br>

Answer the following questions in the context of only the top 15 countries by Scimagojr Rank (aka the DataFrame returned by `answer_one()`)

### Question 3 (6.6%)
What is the average GDP over the last 10 years for each country? (exclude missing values from this calculation.)

*This function should return a Series named `avgGDP` with 15 countries and their average GDP sorted in descending order.*

In [7]:
def answer_three():
    aux = range(2006, 2016)
    fix = [str(i) for i in aux]

    Top15 = answer_one()
    df = Top15[fix]
    df = df.mean(axis=1).sort_values(ascending=False)
    df.name = 'avgGDP'
    return df

answer_three()

Country
United States         1.536434e+13
China                 6.348609e+12
Japan                 5.542208e+12
Germany               3.493025e+12
France                2.681725e+12
United Kingdom        2.487907e+12
Brazil                2.189794e+12
Italy                 2.120175e+12
India                 1.769297e+12
Canada                1.660647e+12
Russian Federation    1.565459e+12
Spain                 1.418078e+12
Australia             1.164043e+12
South Korea           1.106715e+12
Iran                  4.441558e+11
Name: avgGDP, dtype: float64

### Question 4 (6.6%)
By how much had the GDP changed over the 10 year span for the country with the 6th largest average GDP?

*This function should return a single number.*

In [8]:
def answer_four():
    sixth = answer_three().index[5]
    Top15 = answer_one().loc[sixth]
    aux = range(2006, 2016)
    fix = [str(i) for i in aux]
    df = Top15[fix]
    df = df['2015']-df['2006']
    return df.astype(np.float64)

answer_four()

246702696075.3999

### Question 5 (6.6%)
What is the mean `Energy Supply per Capita`?

*This function should return a single number.*

In [9]:
def answer_five():
    Top15 = answer_one()['Energy Supply per Capita']
    return Top15.mean()

answer_five()

157.6

### Question 6 (6.6%)
What country has the maximum % Renewable and what is the percentage?

*This function should return a tuple with the name of the country and the percentage.*

In [10]:
def answer_six():
    Top15 = answer_one()
    Top15 = Top15['% Renewable'].sort_values(ascending=False)
    s = Top15
    return tuple((s.index.values[0], s[0]))

print(answer_six())

('Brazil', 69.648030000000006)


### Question 7 (6.6%)
Create a new column that is the ratio of Self-Citations to Total Citations. 
What is the maximum value for this new column, and what country has the highest ratio?

*This function should return a tuple with the name of the country and the ratio.*

In [11]:
def answer_seven():
    Top15 = answer_one()
    Top15['Ratio'] = Top15['Self-citations']/Top15['Citations']
    Top15 = Top15['Ratio'].sort_values(ascending=False)
    s = Top15
    return tuple((s.index.values[0], s[0]))

answer_seven()

('China', 0.68931261793894216)

### Question 8 (6.6%)

Create a column that estimates the population using Energy Supply and Energy Supply per capita. 
What is the third most populous country according to this estimate?

*This function should return a single string value.*

In [12]:
def answer_eight():
    Top15 = answer_one()
    Top15['Populaiton'] = Top15['Energy Supply'] / Top15['Energy Supply per Capita']
    Top15 = Top15['Populaiton'].sort_values(ascending=False)
    s = Top15
    return s.index.values[2]

answer_eight()

'United States'

### Question 9 (6.6%)
Create a column that estimates the number of citable documents per person. 
What is the correlation between the number of citable documents per capita and the energy supply per capita? Use the `.corr()` method, (Pearson's correlation).

*This function should return a single number.*

*(Optional: Use the built-in function `plot9()` to visualize the relationship between Energy Supply per Capita vs. Citable docs per Capita)*

In [13]:
def answer_nine():
    Top15 = answer_one()
    Top15['PopEst'] = Top15['Energy Supply'].astype(float) / Top15['Energy Supply per Capita'].astype(float)
    Top15['Citable docs per Capita'] = Top15['Citable documents'].astype(float) / Top15['PopEst'].astype(float)
    return Top15.corr()['Energy Supply per Capita']['Citable docs per Capita']

answer_nine()

0.79400104354429435

In [14]:
def plot9():
    import matplotlib as plt
    %matplotlib inline
    
    Top15 = answer_one()
    Top15['PopEst'] = Top15['Energy Supply'] / Top15['Energy Supply per Capita']
    Top15['Citable docs per Capita'] = Top15['Citable documents'] / Top15['PopEst']
    Top15.plot(x='Citable docs per Capita', y='Energy Supply per Capita', kind='scatter', xlim=[0, 0.0006])

In [16]:
#plot9() # Be sure to comment out plot9() before submitting the assignment!

### Question 10 (6.6%)
Create a new column with a 1 if the country's % Renewable value is at or above the median for all countries in the top 15, and a 0 if the country's % Renewable value is below the median.

*This function should return a series named `HighRenew` whose index is the country name sorted in ascending order of rank.*

In [17]:
def answer_ten():
    Top15 = answer_one()
    median = Top15['% Renewable'].median()
    Top15['bin'] = Top15['% Renewable'] >= median
    Top15['bin'] = Top15['bin'].apply(lambda x: int(x))
    series = Top15.sort_values(by='Rank')['bin']
    series.name = 'HighRenew'
    return series

answer_ten()

Country
China                 1
United States         0
Japan                 0
United Kingdom        0
Russian Federation    1
Canada                1
Germany               1
India                 0
France                1
South Korea           0
Italy                 1
Spain                 1
Iran                  0
Australia             0
Brazil                1
Name: HighRenew, dtype: int64

### Question 11 (6.6%)
Use the following dictionary to group the Countries by Continent, then create a dateframe that displays the sample size (the number of countries in each continent bin), and the sum, mean, and std deviation for the estimated population of each country.

```python
ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}
```

*This function should return a DataFrame with index named Continent `['Asia', 'Australia', 'Europe', 'North America', 'South America']` and columns `['size', 'sum', 'mean', 'std']`*

In [18]:
def get_dict():
    return   {'China':'Asia', 
              'United States':'North America', 
              'Japan':'Asia', 
              'United Kingdom':'Europe', 
              'Russian Federation':'Europe', 
              'Canada':'North America', 
              'Germany':'Europe', 
              'India':'Asia',
              'France':'Europe', 
              'South Korea':'Asia', 
              'Italy':'Europe', 
              'Spain':'Europe', 
              'Iran':'Asia',
              'Australia':'Australia', 
              'Brazil':'South America'}

def answer_eleven():
    Top15 = answer_one()
    Top15['PopEst'] = Top15['Energy Supply'].astype(float) / Top15['Energy Supply per Capita'].astype(float)
    grouped = Top15.groupby(by=get_dict())['PopEst'].agg({'size': np.size, 'sum': np.sum, 'mean': np.mean, 'std': np.std})
    grouped.index.names = ['Continent']
    grouped.name = 'Continent'
    return grouped[['size', 'sum', 'mean', 'std']]

answer_eleven()

C:\Users\paula.ceccon.ribeiro\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version


,size,sum,mean,std
Continent,,,,
Asia,5.0,2.898666e+09,5.797333e+08,6.790979e+08
Australia,1.0,2.331602e+07,2.331602e+07,NaN
Europe,6.0,4.579297e+08,7.632161e+07,3.464767e+07
North America,2.0,3.528552e+08,1.764276e+08,1.996696e+08
South America,1.0,2.059153e+08,2.059153e+08,NaN


### Question 12 (6.6%)
Cut % Renewable into 5 bins. Group Top15 by the Continent, as well as these new % Renewable bins. How many countries are in each of these groups?

*This function should return a __Series__ with a MultiIndex of `Continent`, then the bins for `% Renewable`. Do not include groups with no countries.*

In [19]:
def answer_twelve():
    Top15 = answer_one()
    grouped = Top15.groupby([get_dict(), pd.cut(Top15['% Renewable'], 5)]).size()
    grouped.index = grouped.index.set_names('Continent', level=0)
    return grouped

answer_twelve().index

MultiIndex(levels=[['Asia', 'Australia', 'Europe', 'North America', 'South America'], [(2.212, 15.753], (15.753, 29.227], (29.227, 42.701], (42.701, 56.174], (56.174, 69.648]]],
           labels=[[0, 0, 1, 2, 2, 2, 3, 3, 4], [0, 1, 0, 0, 1, 2, 0, 4, 4]],
           names=['Continent', '% Renewable'])

### Question 13 (6.6%)
Convert the Population Estimate series to a string with thousands separator (using commas). Do not round the results.

e.g. 317615384.61538464 -> 317,615,384.61538464

*This function should return a Series `PopEst` whose index is the country name and whose values are the population estimate string.*

In [20]:
def answer_thirteen():
    Top15 = answer_one()
    Top15['PopEst'] = Top15['Energy Supply'].astype(float) / Top15['Energy Supply per Capita'].astype(float)
    Top15['PopEst'] =  Top15['PopEst'].apply(lambda x: '{0:,}'.format(x))
    series = pd.Series(Top15['PopEst'])
    series.name = 'PopEst'
    return series
                                             
answer_thirteen()

Country
China                 1,367,645,161.2903225
United States          317,615,384.61538464
Japan                  127,409,395.97315437
United Kingdom         63,870,967.741935484
Russian Federation            143,500,000.0
Canada                  35,239,864.86486486
Germany                 80,369,696.96969697
India                 1,276,730,769.2307692
France                  63,837,349.39759036
South Korea            49,805,429.864253394
Italy                  59,908,256.880733944
Spain                    46,443,396.2264151
Iran                    77,075,630.25210084
Australia              23,316,017.316017315
Brazil                 205,915,254.23728815
Name: PopEst, dtype: object

### Optional

Use the built in function `plot_optional()` to see an example visualization.

In [21]:
def plot_optional():
    import matplotlib as plt
    %matplotlib inline
    Top15 = answer_one()
    ax = Top15.plot(x='Rank', y='% Renewable', kind='scatter', 
                    c=['#e41a1c','#377eb8','#e41a1c','#4daf4a','#4daf4a','#377eb8','#4daf4a','#e41a1c',
                       '#4daf4a','#e41a1c','#4daf4a','#4daf4a','#e41a1c','#dede00','#ff7f00'], 
                    xticks=range(1,16), s=6*Top15['2014']/10**10, alpha=.75, figsize=[16,6]);

    for i, txt in enumerate(Top15.index):
        ax.annotate(txt, [Top15['Rank'][i], Top15['% Renewable'][i]], ha='center')

    print("This is an example of a visualization that can be created to help understand the data. \
This is a bubble chart showing % Renewable vs. Rank. The size of the bubble corresponds to the countries' \
2014 GDP, and the color corresponds to the continent.")

In [22]:
#plot_optional() # Be sure to comment out plot_optional() before submitting the assignment!